In [225]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

#### Import data

In [226]:
BATCH_SIZE = 6

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


#### Show some images and labels

In [227]:
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(BATCH_SIZE)))

 deer  frog plane   cat  deer horse


#### Neural net definition 

In [321]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

width = 6

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, width, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(width, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
NUM_LAYERS = 3
WIDTH = 64
NUM_CLASSES = 10
IMAGE_DIMENSIONS = 32*32*3
CONV_WIDTH = 18

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device)

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
#         self.fcs = nn.Linear(IMAGE_DIMENSIONS, WIDTH)
#         nn.init.xavier_uniform_(self.fcs.weight)
#         self.fcf = nn.Linear(WIDTH, NUM_CLASSES)
        self.fcf = nn.Linear(CONV_WIDTH*32*32, NUM_CLASSES)
        self.convs = nn.Conv2d(3, CONV_WIDTH, 5, padding=(2, 2))
#         self.fc = nn.ModuleList()
        self.conv = nn.ModuleList()
        for i in range(NUM_LAYERS):
            self.conv.append(nn.Conv2d(CONV_WIDTH, CONV_WIDTH, 5, padding=(2, 2)))
        #    self.fc.append(nn.Linear(WIDTH, WIDTH))
        
        def init_weights(m):
            if type(m) == nn.Conv2d:
                torch.nn.init.xavier_normal_(m.weight, gain=0.01)
                m.bias.data.fill_(0.01)
        self.conv.apply(init_weights)
    
    def forward(self, x):
#         x = x.view(-1, IMAGE_DIMENSIONS)
#         x = F.relu(self.fcs(x))
        x = F.relu(self.convs(x))
        for i in range(NUM_LAYERS):
            #x = x + F.relu(self.fc[i](x))
            x = x + F.relu(self.conv[i](x))
        x = x.view(-1, CONV_WIDTH*32*32)
        x = F.relu(self.fcf(x))
        return x
    

net = ResNet()
print(net.to(device))


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.Adam(net.parameters()) #(net.parameters(), lr=0.001, momentum=0.9)

cuda:0
ResNet(
  (fcf): Linear(in_features=18432, out_features=10, bias=True)
  (convs): Conv2d(3, 18, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv): ModuleList(
    (0): Conv2d(18, 18, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Conv2d(18, 18, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): Conv2d(18, 18, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
)


#### Train the network

In [ ]:
import time
testiter = iter(testloader)
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
for epoch in range(5):  # loop over the dataset multiple times

    start = time.time()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            correct = 0
            total = 0
            with torch.no_grad():
            #     print(time.time())
            #     for i in range(10):
            #         data = testiter.next()
                for data in testloader:
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()


            print(time.time())
            print('Accuracy of the network on the 1000 test images: %d %%' % (
                100 * correct / total))
            
            
            print('Time: ' + str(time.time()-start))
            start = time.time()
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [269]:
print(dataiter.next()[0][0][0][0])


tensor([-0.0902, -0.3098, -0.2863, -0.2627, -0.2392,  0.0667,  0.0667,  0.0510,
        -0.0275, -0.1059, -0.2549, -0.2863, -0.2941, -0.3176, -0.2706, -0.2078,
        -0.2078, -0.2078, -0.2549, -0.2392, -0.2706, -0.2627, -0.2157, -0.2784,
        -0.3098, -0.2941, -0.3412, -0.3569, -0.3804, -0.3882, -0.3569, -0.2471])


#### Show some images and labels

In [232]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

GroundTruth:    cat  ship  ship plane


RuntimeError: Expected object of type torch.cuda.FloatTensor but found type torch.FloatTensor for argument #4 'mat1'

In [315]:
correct = 0
total = 0
with torch.no_grad():
    i = 0
#     print(time.time())
#     for i in range(10):
#         data = testiter.next()
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(time.time())
print('Accuracy of the network on the 1000 test images: %d %%' % (
    100 * correct / total))

1535114819.365087
1535114825.3605514
Accuracy of the network on the 1000 test images: 50 %


In [133]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 32 %
Accuracy of   car : 51 %
Accuracy of  bird : 29 %
Accuracy of   cat : 35 %
Accuracy of  deer : 50 %
Accuracy of   dog : 14 %
Accuracy of  frog : 66 %
Accuracy of horse : 65 %
Accuracy of  ship : 67 %
Accuracy of truck : 58 %
